
<font color="#1874CD"><h1 align="left">**Datamecum**</h1></font>
<font color="#6E6E6E"><h2 align="left">**Árboles de decisión**</h2></font>
<font color="#6E6E6E"><h3 align="left">Notebook: aplicación de las SVM a un problema de clasificación</h3></font>
***

El objetivo de este *notebook* es familiarizarnos con el uso de las SVM en un problema de clasificación. Para ello, haremos uso de un conjunto de datos cuya finalidad es predecir si un individuo tendrá diabetes o no.

Los datos (Pima Indians Diabetes) provienen del Instituto Nacional de Diabetes y de enfermedades del riñón y digestivas.

In [27]:
## ---------------------------------- ##
## Librerías 
## ---------------------------------- ##

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
np.random.seed(19)

%matplotlib inline

## Cargamos los datos

In [ ]:
file = 'data/pimaND.csv'
data = pd.read_csv(file, sep=',')
colnames = data.columns
print(data.shape)
data.head()

## Análisis exploratorio (EDA)

In [ ]:
data.describe().T

## No hay valores ausentes
## Es necesario normalizar/estandarizar las variables

In [ ]:
data.hist(figsize=(15, 15));

In [ ]:
## Podemos ver distribución de cada variable y enfrentada al resto
sns.pairplot(data, hue = 'label');

In [ ]:
## Contamos el target para saber qué proporción tenemos
y_prev = np.array(data['label'])

def show_counts(y):
    uns, counts = np.unique(y, return_counts=True)
    for u,c in zip(uns, counts):
        print("y={}: {} times".format(u, c))

show_counts(y_prev)

plt.figure(figsize=(5,4))
uns,counts = np.unique(y_prev, return_counts=True)
plt.bar(uns, counts)
plt.xticks([0,1])
plt.title("Conteos target")

plt.show()

print(f"Hay un total de {len(y_prev)} registros, de los que el {round(100*counts[1]/len(y_prev), 2)}% corresponden a la clase 1.")

## Separación en train y test

In [32]:
clases = data['label'] 
X = data[colnames[:-1]]

# Separamos en dos conjuntos, training y test:
X_train, X_test, y_train, y_test = train_test_split(X, clases, test_size=0.3, random_state=10, stratify=clases)

In [ ]:
## ¿Mantenemos proporciones?
print(f"Proporción del target en train: {y_train.value_counts() / len(y_train)}")
print(f"Proporción del target en test: {y_test.value_counts() / len(y_test)}")

## Preprocesado de los datos

Para el uso de las SVM, vamos a estandarizar los datos de entrada. Recordar que se hace sobre el **conjunto de entrenamiento** y la transformación resultantes es la que aplicaremos al conjunto de test.

In [34]:
## Estandarizamos los datos, salvo la última columna que es la clase:
sc = StandardScaler()
sc.fit(X_train)
X_train_scal = sc.transform(X_train)
X_test_scal = sc.transform(X_test)

## Optimización de los hiperparámetros

Hacemos un *grid search* para encontrar la mejor combinación de hiperparámetros.

Recordar que valores bajos de C hacen que el margen sea más amplio, es decir, evita sobreajuste. Por otro lado, valores bajos de gamma pueden hacer que el kernel rbf sea lineal, por lo que es bueno que sea el propio *grid search* el que busque el mejor umbral.

In [ ]:
## Construcción del clasificador
grid = [{'C': [1, 10, 100], 
         'gamma': [0.1, 0.01, 0.001], ## 'gamma': "scale" -> ajuste automático!
         'kernel': ['rbf', 'poly'],
         'degree': [2, 3, 4],
         'coef0': [1]
        },
       ]

clf_gridsearch = GridSearchCV(estimator=SVC(probability=True), param_grid=grid, cv=5)
clf_gridsearch.fit(X_train_scal, y_train) 

## Para poder hacer predict_proba necesitamos pasarle probability=True (se indica en la ayuda de sklearn.svc)

In [ ]:
## ------------------------------------------- ##
## Métricas
## ------------------------------------------- ##

## Imprimimos los resultados para el mejor clasificador
print("Best score = ", clf_gridsearch.best_score_)
clf = clf_gridsearch.best_estimator_
print("Best model: ")
print(clf)

## Calculamos el acierto en los conjuntos de entrenamiento y test (accuracy)
score_train = clf.score(X_train_scal, y_train)
print("Score in training set = ", score_train)

score_test = clf.score(X_test_scal, y_test)
print("Score in test set = ", score_test)

## Nota: si el mejor modelo tiene degree = 3, posiblemente no aparezca en los hiperparámetros al ser el valor por defecto

In [ ]:
## ------------------------------------------- ##
## Curva ROC
## ------------------------------------------- ##
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, roc_curve, auc

y_test_proba_predict  = clf.predict_proba(X_test_scal)

fpr, tpr, thresholds = roc_curve(y_test,
                                 y_test_proba_predict[:,1],
                                 pos_label=1)
roc_auc = auc(fpr, tpr)

plt.plot(fpr, tpr, "-o", color='darkorange',
         lw=2, label='model (AUC = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label = 'random')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC at test')
plt.legend(loc="lower right");